In [5]:
import pandas as pd
import sqlite3

In [6]:
df1 = pd.read_csv('player_defense.csv')
df2 = pd.read_csv('player_gsc.csv')
df3 = pd.read_csv('player_misc.csv')
df4 = pd.read_csv('player_passing.csv')
df5 = pd.read_csv('player_possesion.csv')
df6 = pd.read_csv('player_shooting.csv')
df7 = pd.read_csv('player_time.csv')

In [8]:
df1.drop(df1.columns[[8,15, 17, 19]], 1, inplace = True)
df1.drop(['Matches', '-9999'], 1, inplace = True)
df2.drop(['Rk', 'Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'Matches', '-9999', 'SCA', 'SCA90', 'GCA', 'GCA90'], 1, inplace = True)
df3.drop(['Rk', 'Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'Matches', '-9999', 'Int', 'TklW'], 1, inplace = True)
df4.drop(['Rk', 'Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'Matches', '-9999', 'Cmp.1', 'Att.1', 'Cmp%.1', 'Cmp%.2', 'Cmp%.3', 'A-xA', 'KP', '1/3'], 1, inplace = True)
df5.drop(['Rk', 'Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'Matches', '-9999', 'Touches', 'Live', 'Succ%', '1/3'], 1, inplace = True)
df6.drop(['Rk', 'Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'SoT%', 'Sh/90', 'SoT/90', 'G/Sh', 'G/SoT', 'npxG/Sh', 'G-xG', 'np:G-xG'], 1, inplace = True)
df7.drop(['Rk', 'Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'Matches', '-9999', 'Mn/MP', 'Min%', '+/-90', 'xG+/-', 'xG+/-90', 'On-Off.1'], 1, inplace = True)

C:\Users\Brian\AppData\Local\Temp/ipykernel_21040/3956993825.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df1.drop(df1.columns[[8,15, 17, 19]], 1, inplace = True)
C:\Users\Brian\AppData\Local\Temp/ipykernel_21040/3956993825.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df1.drop(['Matches', '-9999'], 1, inplace = True)
C:\Users\Brian\AppData\Local\Temp/ipykernel_21040/3956993825.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df2.drop(['Rk', 'Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'Matches', '-9999', 'SCA', 'SCA90', 'GCA', 'GCA90'], 1, inplace = True)
C:\Users\Brian\AppData\Local\Temp/ipykernel_21040/3956993825.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for th

In [9]:
df = pd.merge(df1, df2, how = 'inner', on='Player')
df = pd.merge(df, df3, how = 'inner', on='Player')
df = pd.merge(df, df4, how = 'inner', on='Player')
df = pd.merge(df, df5, how = 'inner', on='Player')
df = pd.merge(df, df6, how = 'inner', on='Player')
df = pd.merge(df, df7, how = 'inner', on='Player')

In [11]:
df['Nation'] = list(map(lambda x: x[-3:], df['Nation']))
df['Age'] = list(map(lambda x: x[:2], df['Age']))
df.drop('Rk', 1, inplace = True)
df.reset_index(inplace = True)
df['index'] = list(map(lambda x: x+1, df['index']))

C:\Users\Brian\AppData\Local\Temp/ipykernel_21040/1613061821.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop('Rk', 1, inplace = True)


In [13]:
players = df[['index', 'Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born']]
players.rename(columns = {'index':'Id'}, inplace = True)
stats = df.drop(['Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born'], 1)
stats.rename(columns = {'index':'PlayerId'}, inplace = True)

C:\Users\Brian\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
C:\Users\Brian\AppData\Local\Temp/ipykernel_21040/951836375.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  stats = df.drop(['Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born'], 1)


In [15]:
#connect to mlsdb and create/update players table
with sqlite3.connect('mlsdb.sqlite') as conn:
    c = conn.cursor()
    c.execute('CREATE TABLE IF NOT EXISTS players (id INT, name TEXT, nation TEXT, position TEXT, squad TEXT, age INTEGER, born INTEGER)')
    conn.commit()
    players.to_sql('players', conn, if_exists='replace', index = False)
    
    c.execute('CREATE TABLE IF NOT EXISTS stats (playerId INT, ninetys FLOAT, tklW INT, tklDef INT, tklMid INT, tklAtt INT, tklDrib INT, att INT, past INT, pressW INT, pressDef INT, pressMid INT, pressAtt INT, shotsBlocked INT, passesBlocked INT, int INT, clr INT, err INT, scPassLive INT, scPassDead INT, scDrib INT, scSh INT, scFld INT, scDef INT, gcPassLive INT, gcPassDead INT, gcDrib INT, gcSh INT, gcFld INT, gcDef INT, crdY INT, crdR INT, crdR2 INT, fls INT, fld INT, off INT, crs INT, pkEarned INT, pkGiven INT, og INT, recov INT, airW INT, airL INT, passTotDist INT, passPrgDist INT, shortPassCmp INT, shortPassAtt INT, medPassCmp INT, medPassAtt INT, lngPassCmp INT, lngPassAtt INT, ast INT, xa INT, ppa INT, crspa INT, progPasses INT, defPenTouches INT, defTouches INT, midTouches INT, attTouches INT, attPenTouches INT, dribSucc INT, dribAtt INT, dribPast INT, megs INT, carries INT, carriesDist INT, carriesPrgDist INT, cariesProg INT, carriesPa INT, carriesMis INT, carriesLost INT, passTarget INT, passReciever INT, recievePct FLOAT, progPassRecieved INT, gls INT, sh INT, shOnTarget INT, avgShDist FLOAT, fkSh INT, pkMade INT, pkAtt INT, xg INT, npxg INT, matches INT, min INT, starts INT, minPerStart INT, complete INT, subs INT, minPerSub INT, unsub INT, ppm FLOAT, goalsOnPitch INT, gaOnPitch INT, plusMinus INT, onOff FLOAT, onXg FLOAT, onXga FLOAT)')     
    conn.commit()
    stats.to_sql('stats', conn, if_exists='replace', index = False)    

C:\Users\Brian\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


Index(['PlayerId', 'Rk_x', 'Nation_x', 'Pos_x', 'Squad_x', 'Age_x', 'Born_x',
       '90s_x', 'Tkl', 'TklW_x',
       ...
       '+/-', '+/-90', 'On-Off', 'onxG', 'onxGA', 'xG+/-', 'xG+/-90',
       'On-Off.1', 'Matches_y', '-9999_y'],
      dtype='object', length=195)